# データサイエンス100本ノック（構造化データ加工編） - SQL

## はじめに
- データベースはPostgreSQL12です
- 初めに以下のセルを実行してください
- セルに %%sql と記載することでSQLを発行することができます
- jupyterからはdescribeコマンドによるテーブル構造の確認ができないため、テーブル構造を確認する場合はlimitを指定したSELECTなどで代用してください
- 使い慣れたSQLクライアントを使っても問題ありません（接続情報は以下の通り）
  - IPアドレス：Docker Desktopの場合はlocalhost、Docker toolboxの場合は192.168.99.100
  - Port:5432
  - database名: dsdojo_db
  - ユーザ名：padawan
  - パスワード:padawan12345
- 大量出力を行うとJupyterが固まることがあるため、出力件数は制限することを推奨します（設問にも出力件数を記載）
    - 結果確認のために表示させる量を適切にコントロールし、作業を軽快にすすめる技術もデータ加工には求められます
- 大量結果が出力された場合は、ファイルが重くなり以降開けなくなることもあります
    - その場合、作業結果は消えますがファイルをGitHubから取り直してください
    - vimエディタなどで大量出力範囲を削除することもできます
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
%load_ext sql
import os
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# pd.read_sql用のコネクタ
conn = psycopg2.connect(**pgconfig)
# pd.to_sql用のcreate engine
engine = create_engine(dsl)
# MagicコマンドでSQLを書くための設定
%sql $dsl

'Connected: padawan@dsdojo_db'

# 使い方
- セルの先頭に%%sqlと記載し、２行目以降にSQLを記述することでJupyterからPostgreSQLに対しSQLを実行できます。

In [ ]:
%%sql
select 'このように実行できます' as sample

# データ加工100本ノック

---
> S-001: レシート明細テーブル（receipt）から全項目を10件抽出し、どのようなデータを保有しているか目視で確認せよ。


In [2]:
%%sql
SELECT * FROM receipt LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> S-002: レシート明細のテーブル（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [6]:
%%sql
SELECT sales_ymd, customer_id, product_cd, amount FROM receipt LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,customer_id,product_cd,amount
20181103,CS006214000001,P070305012,158
20181118,CS008415000097,P070701017,81
20170712,CS028414000014,P060101005,170
20190205,ZZ000000000000,P050301001,25
20180821,CS025415000050,P060102007,90
20190605,CS003515000195,P050102002,138
20181205,CS024514000042,P080101005,30
20190922,CS040415000178,P070501004,128
20170504,ZZ000000000000,P071302010,770
20191010,CS027514000015,P071101003,680


---
> S-003: レシート明細のテーブル（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [610]:
%%sql
SELECT sales_ymd AS sales_date, customer_id, product_cd, amount FROM receipt LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_date,customer_id,product_cd,amount
20181103,CS006214000001,P070305012,158
20181118,CS008415000097,P070701017,81
20170712,CS028414000014,P060101005,170
20190205,ZZ000000000000,P050301001,25
20180821,CS025415000050,P060102007,90
20190605,CS003515000195,P050102002,138
20181205,CS024514000042,P080101005,30
20190922,CS040415000178,P070501004,128
20170504,ZZ000000000000,P071302010,770
20191010,CS027514000015,P071101003,680


---
> S-004: レシート明細のテーブル（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ
> - 顧客ID（customer_id）が"CS018205000001"

In [11]:
%%sql
SELECT sales_ymd, customer_id, product_cd, amount FROM receipt
WHERE customer_id = 'CS018205000001'

 * postgresql://padawan:***@db:5432/dsdojo_db
12 rows affected.


sales_date,customer_id,product_cd,amount
20180911,CS018205000001,P071401012,2200
20180414,CS018205000001,P060104007,600
20170614,CS018205000001,P050206001,990
20170614,CS018205000001,P060702015,108
20190216,CS018205000001,P071005024,102
20180414,CS018205000001,P071101002,278
20190226,CS018205000001,P070902035,168
20190924,CS018205000001,P060805001,495
20190226,CS018205000001,P071401020,2200
20180911,CS018205000001,P071401005,1100


---
> S-005: レシート明細のテーブル（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [2]:
%%sql
SELECT sales_ymd , customer_id, product_cd, amount FROM receipt
WHERE customer_id = 'CS018205000001' AND amount >= 1000

 * postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401012,2200
20190226,CS018205000001,P071401020,2200
20180911,CS018205000001,P071401005,1100


---
> S-006: レシート明細テーブル（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [3]:
%%sql
SELECT sales_ymd, customer_id, product_cd, quantity, amount FROM receipt
WHERE customer_id = 'CS018205000001' AND (amount >= 1000 OR quantity >=5)

 * postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


sales_ymd,customer_id,product_cd,quantity,amount
20180911,CS018205000001,P071401012,1,2200
20180414,CS018205000001,P060104007,6,600
20170614,CS018205000001,P050206001,5,990
20190226,CS018205000001,P071401020,1,2200
20180911,CS018205000001,P071401005,1,1100


---
> S-007: レシート明細のテーブル（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [15]:
%%sql
SELECT sales_ymd, customer_id, product_cd, amount FROM receipt
WHERE customer_id = 'CS018205000001' AND amount BETWEEN 1000 AND 2000

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401005,1100


---
> S-008: レシート明細テーブル（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [17]:
%%sql
SELECT sales_ymd, customer_id, product_cd, amount FROM receipt
WHERE customer_id = 'CS018205000001' AND product_cd <> 'P071401019'

 * postgresql://padawan:***@db:5432/dsdojo_db
12 rows affected.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401012,2200
20180414,CS018205000001,P060104007,600
20170614,CS018205000001,P050206001,990
20170614,CS018205000001,P060702015,108
20190216,CS018205000001,P071005024,102
20180414,CS018205000001,P071101002,278
20190226,CS018205000001,P070902035,168
20190924,CS018205000001,P060805001,495
20190226,CS018205000001,P071401020,2200
20180911,CS018205000001,P071401005,1100


---
> S-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`select * from store where not (prefecture_cd = '13' or floor_area > 900)`

In [5]:
%%sql
SELECT * FROM store where NOT prefecture_cd = '13' AND NOT floor_area > 900

 * postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.022,35.66122,808.0


---
> S-010: 店舗テーブル（store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [613]:
%%sql
SELECT * FROM store
WHERE store_cd ~ '^S14.*'
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> S-011: 顧客テーブル（customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。


In [614]:
%%sql
SELECT * FROM customer
WHERE customer_id ~ '.*1$'
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS037613000071,六角 雅彦,9,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS028811000001,堀井 かおり,1,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
CS040412000191,川井 郁恵,1,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
CS028314000011,小菅 あおい,1,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
CS039212000051,藤島 恵梨香,1,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
CS015412000111,松居 奈月,1,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
CS004702000041,野島 洋,0,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
CS041515000001,栗田 千夏,1,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
CS029313000221,北条 ひかり,1,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
CS034312000071,望月 奈央,1,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> S-012: 店舗テーブル（store）から横浜市の店舗だけ全項目表示せよ。

In [615]:
%%sql
SELECT * FROM store
WHERE address ~ '横浜市'

 * postgresql://padawan:***@db:5432/dsdojo_db
11 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> S-013: 顧客テーブル（customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [616]:
%%sql
SELECT * FROM customer
WHERE status_cd ~ '^[A-F].*'
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS031415000172,宇多田 貴美子,1,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
CS015414000103,奥野 陽子,1,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
CS011215000048,芦田 沙耶,1,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
CS029415000023,梅田 里穂,1,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
CS035415000029,寺沢 真希,9,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
CS031415000106,宇野 由美子,1,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
CS029215000025,石倉 美帆,1,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
CS033605000005,猪股 雄太,0,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
CS033415000229,板垣 菜々美,1,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
CS008415000145,黒谷 麻緒,1,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> S-014: 顧客テーブル（customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [617]:
%%sql
SELECT * FROM customer
WHERE status_cd ~ '.*[1-9]$'
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS001215000145,田崎 美紀,1,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
CS033513000180,安斎 遥,1,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
CS011215000048,芦田 沙耶,1,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
CS040412000191,川井 郁恵,1,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
CS009315000023,皆川 文世,1,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
CS015315000033,福士 璃奈子,1,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
CS023513000066,神戸 そら,1,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
CS035513000134,市川 美帆,1,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
CS001515000263,高松 夏空,1,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
CS040314000027,鶴田 きみまろ,9,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


---
> S-015: 顧客テーブル（customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [618]:
%%sql
SELECT * FROM customer
WHERE status_cd ~ '^[A-F].*[1-9]$'
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS011215000048,芦田 沙耶,1,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
CS022513000105,島村 貴美子,1,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
CS001515000096,水野 陽子,9,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
CS013615000053,西脇 季衣,1,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
CS020412000161,小宮 薫,1,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
CS001215000097,竹中 あさみ,1,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
CS035212000007,内村 恵梨香,1,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
CS002515000386,野田 コウ,1,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
CS001615000372,稲垣 寿々花,1,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
CS032512000121,松井 知世,1,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> S-016: 店舗テーブル（store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [29]:
%%sql
SELECT * FROM store
WHERE tel_no ~ '^[0-9]{3}-[0-9]{3}-[0-9]{4}$'

 * postgresql://padawan:***@db:5432/dsdojo_db
34 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698.0
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


---
> S-017: 顧客テーブル（customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [21]:
%%sql
SELECT * FROM customer
ORDER BY birth_day ASC
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0
CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0


---
> S-018: 顧客テーブル（customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [22]:
%%sql
SELECT * FROM customer
ORDER BY birth_day DESC
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> S-019: レシート明細テーブル（receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [24]:
%%sql
SELECT customer_id, amount, RANK() OVER (ORDER BY amount DESC) FROM receipt
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,amount,rank
CS011415000006,10925,1
ZZ000000000000,6800,2
CS028605000002,5780,3
ZZ000000000000,5480,4
CS015515000034,5480,4
ZZ000000000000,5480,4
ZZ000000000000,5440,7
CS021515000089,5440,7
ZZ000000000000,5280,9
CS009415000038,5280,9


---
>  S-020: レシート明細テーブル（receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [25]:
%%sql
SELECT customer_id, amount, ROW_NUMBER() OVER (ORDER BY amount DESC) FROM receipt
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,amount,row_number
CS011415000006,10925,1
ZZ000000000000,6800,2
CS028605000002,5780,3
ZZ000000000000,5480,4
CS015515000034,5480,5
ZZ000000000000,5480,6
ZZ000000000000,5440,7
CS021515000089,5440,8
ZZ000000000000,5280,9
CS009415000038,5280,10


---
> S-021: レシート明細テーブル（receipt）に対し、件数をカウントせよ。

In [26]:
%%sql
SELECT COUNT(*) FROM receipt

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


count
104681


---
> S-022: レシート明細テーブル（receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [28]:
%%sql
SELECT COUNT(DISTINCT customer_id) FROM receipt

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


count
8307


---
> S-023: レシート明細テーブル（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [619]:
%%sql
SELECT store_cd, SUM(amount) AS 合計金額, SUM(quantity) AS 合計数量 FROM receipt
GROUP BY store_cd

 * postgresql://padawan:***@db:5432/dsdojo_db
52 rows affected.


store_cd,合計金額,合計数量
S12007,638761,2099
S13017,748221,2376
S13043,587895,1881
S13052,100314,250
S13016,793773,2432
S14027,714550,2303
S13009,808870,2486
S14022,651328,2047
S13019,827833,2541
S13039,611888,1981


---
> S-024: レシート明細テーブル（receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [34]:
%%sql
SELECT customer_id, MAX(sales_ymd) FROM receipt
GROUP BY customer_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,max
CS001311000059,20180211
CS004614000122,20181228
CS003512000043,20180106
CS011615000061,20190503
CS029212000033,20180621
CS007515000119,20190511
CS034515000123,20190708
CS004315000058,20170517
CS026414000014,20190720
CS001615000099,20170729


---
> S-025: レシート明細テーブル（receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [35]:
%%sql
SELECT customer_id, MIN(sales_ymd) FROM receipt
GROUP BY customer_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,min
CS001311000059,20180211
CS004614000122,20181228
CS003512000043,20180106
CS011615000061,20190503
CS029212000033,20170318
CS007515000119,20170201
CS034515000123,20170527
CS004315000058,20170517
CS026414000014,20170718
CS001615000099,20170729


---
> S-026: レシート明細テーブル（receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [37]:
%%sql
SELECT customer_id, MAX(sales_ymd), MIN(sales_ymd) FROM receipt
GROUP BY customer_id
HAVING MAX(sales_ymd) <> MIN(sales_ymd)
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,max,min
CS029212000033,20180621,20170318
CS007515000119,20190511,20170201
CS034515000123,20190708,20170527
CS026414000014,20190720,20170718
CS010515000082,20181204,20180518
CS019315000045,20170920,20170423
CS008513000099,20190308,20170722
CS007615000070,20191025,20170929
CS025415000155,20191026,20170314
CS016414000063,20190617,20170109


---
> S-027: レシート明細テーブル（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [41]:
%%sql
SELECT store_cd, AVG(amount) FROM receipt
GROUP BY store_cd
ORDER BY AVG(amount) DESC
LIMIT 5

 * postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,avg
S13052,402.8674698795180723
S13015,351.1119604316546763
S13003,350.9155188246097337
S14010,348.7912621359223301
S13001,348.4703862660944206


---
> S-028: レシート明細テーブル（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [622]:
%%sql
SELECT store_cd, PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY amount) AS 中央値 FROM receipt
GROUP BY store_cd
ORDER BY PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY amount) DESC
LIMIT 5


 * postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,中央値
S13052,190.0
S14010,188.0
S14050,185.0
S13003,180.0
S13018,180.0


---
> S-029: レシート明細テーブル（receipt）に対し、店舗コード（store_cd）ごとに商品コードの最頻値を求めよ。

In [16]:
%%sql
SELECT store_cd, MODE() WITHIN GROUP (ORDER BY product_cd) FROM receipt
GROUP BY store_cd


 * postgresql://padawan:***@db:5432/dsdojo_db
52 rows affected.


store_cd,mode
S12007,P060303001
S12013,P060303001
S12014,P060303001
S12029,P060303001
S12030,P060303001
S13001,P060303001
S13002,P060303001
S13003,P071401001
S13004,P060303001
S13005,P040503001


---
> S-030: レシート明細テーブル（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順にTOP5を表示せよ。

In [19]:
%%sql
SELECT store_cd, VAR_SAMP(amount) FROM receipt
-- 母集団分散を計算するときはVAR_POPを使う。VARIANCEはVAR_SAMPと同様。（古いもの）
GROUP BY store_cd
ORDER BY VAR_SAMP(amount) DESC
LIMIT 5

 * postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,var_samp
S13052,441863.252526233968
S14011,306442.242431568709
S14034,297068.392740060738
S13001,295558.842617712478
S13015,295427.197085853584


---
> S-031: レシート明細テーブル（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順にTOP5を表示せよ。

In [20]:
%%sql
SELECT store_cd, STDDEV(amount) FROM receipt
GROUP BY store_cd
ORDER BY STDDEV(amount) DESC
LIMIT 5

 * postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,stddev
S13052,664.727953772244
S14011,553.572255836190
S14034,545.039808399406
S13001,543.653237475610
S13015,543.532149082144


---
> S-032: レシート明細テーブル（receipt）に対し、売上金額（amount）について25％刻みでパーセンタイル値を求めよ。

In [623]:
%%sql
SELECT
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY amount) AS Q1,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY amount) AS Q2,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY amount) AS Q3
FROM receipt

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


q1,q2,q3
102.0,170.0,288.0


---
> S-033: レシート明細テーブル（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [16]:
%%sql
SELECT store_cd, AVG(amount) FROM receipt
GROUP BY store_cd
HAVING AVG(amount) >=330

 * postgresql://padawan:***@db:5432/dsdojo_db
13 rows affected.


store_cd,avg
S13052,402.8674698795180723
S13019,330.2086158755484643
S13003,350.9155188246097337
S14045,330.0820734341252700
S13004,330.9439490445859873
S13001,348.4703862660944206
S14026,332.3405884723901653
S14010,348.7912621359223301
S13015,351.1119604316546763
S12013,330.1941299790356394


---
> S-034: レシート明細テーブル（receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [624]:
%%sql
SELECT AVG(sum_amount) FROM (
    SELECT SUM(amount) AS sum_amount FROM receipt WHERE customer_id !~ '^Z.*' GROUP BY customer_id
) AS member

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


avg
2547.7422345292559595


---
> S-035: レシート明細テーブル（receipt）に対し、顧客ID（customer_id）ごとに販売金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [627]:
%%sql
-- receiptから非会員を除外し、amountをcustomer_idごとに合計する。(sum_amount) WITH句にて一度保留。
WITH member_table AS (
    SELECT customer_id, SUM(amount) AS sum_amount FROM receipt WHERE customer_id !~ '^Z.*' GROUP BY customer_id
)
SELECT customer_id FROM member_table
-- AVG(sum_amount)にて全会員の合計購入金額の平均を求める。
WHERE sum_amount >= (SELECT AVG(sum_amount) FROM member_table)
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id
CS029212000033
CS007515000119
CS034515000123
CS026414000014
CS007615000070
CS016414000063
CS012514000018
CS029515000142
CS015215000021
CS039814000011


---
> S-036: レシート明細テーブル（receipt）と店舗テーブル（store）を内部結合し、レシート明細テーブルの全項目と店舗テーブルの店舗名（store_name）を10件表示させよ。

In [628]:
%%sql
SELECT receipt.*, store_name FROM receipt
INNER JOIN store ON receipt.store_cd = store.store_cd
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81,成城店
20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170,二ツ橋店
20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25,新山下店
20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90,大和店
20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138,狛江店
20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30,三田店
20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128,長津田店
20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770,十条仲原店
20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680,南藤沢店


---
> S-037: 商品テーブル（product）とカテゴリテーブル（category）を内部結合し、商品テーブルの全項目とカテゴリテーブルの小区分名（category_small_name）を10件表示させよ。

In [52]:
%%sql
SELECT product.*, category_small_name 
FROM product
INNER JOIN category ON product.category_small_cd = category.category_small_cd
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
P040101001,04,0401,040101,198,149,弁当類
P040101002,04,0401,040101,218,164,弁当類
P040101003,04,0401,040101,230,173,弁当類
P040101004,04,0401,040101,248,186,弁当類
P040101005,04,0401,040101,268,201,弁当類
P040101006,04,0401,040101,298,224,弁当類
P040101007,04,0401,040101,338,254,弁当類
P040101008,04,0401,040101,420,315,弁当類
P040101009,04,0401,040101,498,374,弁当類
P040101010,04,0401,040101,580,435,弁当類


---
> S-038: 顧客テーブル（customer）とレシート明細テーブル（receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [629]:
%%sql
SELECT 
    customer.customer_id, 
    SUM(CASE WHEN amount IS NULL THEN 0 ELSE amount END)
FROM customer
LEFT OUTER JOIN receipt ON customer.customer_id = receipt.customer_id
WHERE gender_cd = '1' AND customer.customer_id !~ '^Z.*'
GROUP BY customer.customer_id
LIMIT 10


 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum
CS004412000425,0
CS007512000199,0
CS002712000064,0
CS039515000243,0
CS001615000099,768
CS010515000082,1482
CS019315000045,813
CS008513000099,1322
CS007615000070,2975
CS029511000005,0


---
> S-039: レシート明細テーブル（receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [89]:
%%sql
WITH freq20_mem AS (
    SELECT customer_id FROM receipt WHERE customer_id !~ '^Z.*' GROUP BY customer_id ORDER BY COUNT(sales_ymd) DESC LIMIT 20
),
     high20_mem AS (
SELECT customer_id FROM receipt WHERE customer_id !~ '^Z.*' GROUP BY customer_id ORDER BY SUM(amount) DESC LIMIT 20
)

SELECT freq20_mem.customer_id AS freq_20, high20_mem.customer_id AS high_20
FROM freq20_mem FULL OUTER JOIN high20_mem ON freq20_mem.customer_id = high20_mem.customer_id

 * postgresql://padawan:***@db:5432/dsdojo_db
34 rows affected.


freq_20,high_20
CS040214000008,None
CS010214000010,CS010214000010
CS015415000185,CS015415000185
CS028415000007,CS028415000007
CS010214000002,None
CS016415000141,CS016415000141
CS021514000045,None
CS007515000107,None
CS017415000097,CS017415000097
CS031414000051,CS031414000051


---
> S-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（store）と商品（product）を直積した件数を計算せよ。

In [91]:
%%sql
SELECT COUNT(*) FROM (SELECT * FROM store CROSS JOIN product) AS c_join

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


count
531590


---
> S-041: レシート明細テーブル（receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [631]:
%%sql
SELECT 
    sales_ymd, 
-- ウィンドウ関数にて1日前の合計金額を取得。
    sum_amount - MIN(sum_amount) OVER (ORDER BY sales_ymd DESC ROWS BETWEEN 1 FOLLOWING AND 1 FOLLOWING) AS 売上金額増減
FROM(
    SELECT sales_ymd, SUM(amount) sum_amount
    FROM receipt
    GROUP BY sales_ymd
) AS sum_amount_table
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,売上金額増減
20191031,-1386
20191030,-9489
20191029,-4070
20191028,2677
20191027,-14287
20191026,22938
20191025,-3035
20191024,2431
20191023,-2308
20191022,-12059


---
> S-042: レシート明細テーブル（receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [633]:
%%sql
SELECT 
    sales_ymd, 
    sum_amount,
    MIN(sum_amount) OVER (ORDER BY sales_ymd DESC ROWS BETWEEN 1 FOLLOWING AND 1 FOLLOWING) AS "d1",
    MIN(sum_amount) OVER (ORDER BY sales_ymd DESC ROWS BETWEEN 2 FOLLOWING AND 2 FOLLOWING) AS "d2",
    MIN(sum_amount) OVER (ORDER BY sales_ymd DESC ROWS BETWEEN 3 FOLLOWING AND 3 FOLLOWING) AS "d3"
FROM (
    SELECT sales_ymd, SUM(amount) AS sum_amount
    FROM receipt
    GROUP BY sales_ymd
) AS sum_amount_table
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,sum_amount,d1,d2,d3
20191031,25216,26602,36091,40161
20191030,26602,36091,40161,37484
20191029,36091,40161,37484,51771
20191028,40161,37484,51771,28833
20191027,37484,51771,28833,31868
20191026,51771,28833,31868,29437
20191025,28833,31868,29437,31745
20191024,31868,29437,31745,43804
20191023,29437,31745,43804,38088
20191022,31745,43804,38088,39025


---
> S-043: レシート明細テーブル（receipt）と顧客テーブル（customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリテーブル（sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
>ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [22]:
%%sql
SELECT 
    TRUNC(age, -1) AS 年代,
    SUM(CASE WHEN gender_cd = '0' THEN amount ELSE 0 END) AS "0",
    SUM(CASE WHEN gender_cd = '1' THEN amount ELSE 0 END) AS "1",
-- NULLを拾う。
    SUM(CASE WHEN gender_cd = '9' THEN amount WHEN gender_cd IS NULL THEN amount ELSE 0 END) AS "9"
FROM receipt FULL OUTER JOIN customer ON receipt.customer_id = customer.customer_id
GROUP BY TRUNC(age, -1)
ORDER BY TRUNC(age, -1)

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


年代,0,1,9
10,1591,149836,4317
20,72940,1363724,44328
30,177322,693047,50441
40,19355,9320791,483512
50,54320,6685192,342923
60,272469,987741,71418
70,13435,29764,2427
80,46360,262923,5111
90,0,6260,0
None,0,0,12395003


---
> S-044: 前設問で作成した売上サマリテーブル（sales_summary）は性別の売上を横持ちさせたものであった。このテーブルから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [25]:
%%sql
SELECT 
    TRUNC(age, -1) AS 年代,
    CASE WHEN gender_cd = '0' THEN '00' WHEN gender_cd = '1' THEN '01' WHEN gender_cd = '9' THEN '99' ELSE NULL END AS 性別コード,
    SUM(amount)
FROM receipt FULL OUTER JOIN customer ON receipt.customer_id = customer.customer_id
GROUP BY TRUNC(age, -1), gender_cd
ORDER BY TRUNC(age, -1), 性別コード ASC

 * postgresql://padawan:***@db:5432/dsdojo_db
27 rows affected.


年代,性別コード,sum
10,00,1591
10,01,149836
10,99,4317
20,00,72940
20,01,1363724
20,99,44328
30,00,177322
30,01,693047
30,99,50441
40,00,19355


---
> S-045: 顧客テーブル（customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [635]:
%%sql
SELECT customer_id, REPLACE(CAST(birth_day AS varchar), '-', '')  AS birth_day
FROM customer
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,birth_day
CS021313000114,19810429
CS037613000071,19520401
CS031415000172,19761004
CS028811000001,19330327
CS001215000145,19950329
CS020401000016,19740915
CS015414000103,19770809
CS029403000008,19730817
CS015804000004,19310502
CS033513000180,19620711


---
> S-046: 顧客テーブル（customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [636]:
%%sql
SELECT customer_id, CAST(application_date AS date)
FROM customer
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,application_date
CS021313000114,2015-09-05
CS037613000071,2015-04-14
CS031415000172,2015-05-29
CS028811000001,2016-01-15
CS001215000145,2017-06-05
CS020401000016,2015-02-25
CS015414000103,2015-07-22
CS029403000008,2015-05-15
CS015804000004,2015-06-07
CS033513000180,2015-07-28


---
> S-047: レシート明細テーブル（receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [109]:
%%sql
SELECT receipt_no, receipt_sub_no, CAST(CAST(sales_ymd AS varchar) AS date)
FROM receipt
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


receipt_no,receipt_sub_no,sales_ymd
112,1,2018-11-03
1132,2,2018-11-18
1102,1,2017-07-12
1132,1,2019-02-05
1102,2,2018-08-21
1112,1,2019-06-05
1102,2,2018-12-05
1102,1,2019-09-22
1112,2,2017-05-04
1102,1,2019-10-10


---
> S-048: レシート明細テーブル（receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（timestamp型）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [114]:
%%sql
SELECT receipt_no, receipt_sub_no, TO_TIMESTAMP(sales_epoch)
FROM receipt
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


receipt_no,receipt_sub_no,to_timestamp
112,1,2018-11-03 00:00:00+00:00
1132,2,2018-11-18 00:00:00+00:00
1102,1,2017-07-12 00:00:00+00:00
1132,1,2019-02-05 00:00:00+00:00
1102,2,2018-08-21 00:00:00+00:00
1112,1,2019-06-05 00:00:00+00:00
1102,2,2018-12-05 00:00:00+00:00
1102,1,2019-09-22 00:00:00+00:00
1112,2,2017-05-04 00:00:00+00:00
1102,1,2019-10-10 00:00:00+00:00


---
> S-049: レシート明細テーブル（receipt）の販売エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [639]:
%%sql
SELECT receipt_no, receipt_sub_no, CAST(EXTRACT(year FROM TO_TIMESTAMP(sales_epoch)) AS integer) AS 年
FROM receipt
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


receipt_no,receipt_sub_no,年
112,1,2018
1132,2,2018
1102,1,2017
1132,1,2019
1102,2,2018
1112,1,2019
1102,2,2018
1102,1,2019
1112,2,2017
1102,1,2019


---
> S-050: レシート明細テーブル（receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [640]:
%%sql
SELECT receipt_no, receipt_sub_no, SUBSTR(CAST(TO_TIMESTAMP(sales_epoch) AS varchar),6 ,2) AS 月
FROM receipt
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


receipt_no,receipt_sub_no,月
112,1,11
1132,2,11
1102,1,07
1132,1,02
1102,2,08
1112,1,06
1102,2,12
1102,1,09
1112,2,05
1102,1,10


---
> S-051: レシート明細テーブル（receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [641]:
%%sql
SELECT receipt_no, receipt_sub_no, SUBSTR(CAST(TO_TIMESTAMP(sales_epoch) AS varchar),9 ,2) AS 日
FROM receipt
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


receipt_no,receipt_sub_no,日
112,1,03
1132,2,18
1102,1,12
1132,1,05
1102,2,21
1112,1,05
1102,2,05
1102,1,22
1112,2,04
1102,1,10


---
> S-052: レシート明細テーブル（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、合計金額とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [642]:
%%sql
SELECT customer_id, SUM(amount), CASE WHEN SUM(amount) <= 2000 THEN 0 ELSE 1 END
FROM (SELECT * FROM receipt WHERE customer_id !~ '^Z.*') AS mem_table
GROUP BY customer_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum,case
CS001311000059,2302,1
CS004614000122,248,0
CS003512000043,298,0
CS011615000061,246,0
CS029212000033,3604,1
CS007515000119,7157,1
CS034515000123,3699,1
CS004315000058,490,0
CS026414000014,6671,1
CS001615000099,768,0


---
> S-053: 顧客テーブル（customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細テーブル（receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [643]:
%%sql
SELECT 
    SUM(CASE WHEN flag_tokyo = 1 THEN flag_purchase ELSE 0 END) AS Tokyo,
    SUM(CASE WHEN flag_tokyo = 0 THEN flag_purchase ELSE 0 END) AS Others
FROM(
-- 郵便番号が100〜209の間かどうか(flag_tokyo)、およびsales_ymdがNULLかどうか(flag_purchase)を判定。
SELECT
    CASE WHEN CAST(LEFT(postal_cd, 3) AS integer) >= 100 THEN CASE WHEN CAST(LEFT(postal_cd, 3) AS integer) <= 209 THEN 1 ELSE 0 END ELSE 0 END AS flag_tokyo,
    CASE WHEN sales_ymd IS NOT NULL THEN 1 ELSE 0 END AS flag_purchase
    FROM customer FULL OUTER JOIN receipt ON customer.customer_id = receipt.customer_id
) AS flag_table

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


tokyo,others
31244,73437


---
> S-054: 顧客テーブル（customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [647]:
%%sql
SELECT customer_id, address,
    CASE LEFT(address, 3)
        WHEN '埼玉県' THEN 11
        WHEN '千葉県' THEN 12
        WHEN '東京都' THEN 13
        WHEN '神奈川' THEN 14
    ELSE 99 END AS 都道府県コード
FROM customer
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,address,都道府県コード
CS021313000114,神奈川県伊勢原市粟窪**********,14
CS037613000071,東京都江東区南砂**********,13
CS031415000172,東京都渋谷区代々木**********,13
CS028811000001,神奈川県横浜市泉区和泉町**********,14
CS001215000145,東京都大田区仲六郷**********,13
CS020401000016,東京都板橋区若木**********,13
CS015414000103,東京都江東区北砂**********,13
CS029403000008,千葉県浦安市海楽**********,12
CS015804000004,東京都江東区北砂**********,13
CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> S-055: レシート明細テーブル（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [648]:
%%sql
SELECT customer_id, sum_amount, 
    CASE
        WHEN PERCENT_RANK() OVER (ORDER BY sum_amount) < 0.25 THEN 1
        WHEN PERCENT_RANK() OVER (ORDER BY sum_amount) < 0.5 THEN 2
        WHEN PERCENT_RANK() OVER (ORDER BY sum_amount) < 0.75 THEN 3
        WHEN PERCENT_RANK() OVER (ORDER BY sum_amount) >= 0.75 THEN 4
    ELSE 0 END AS カテゴリ値
FROM (
    SELECT customer_id, SUM(amount) AS sum_amount
    FROM receipt
    GROUP BY customer_id
) AS sum_amount_table
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,カテゴリ値
CS014614000056,70,1
CS002513000267,80,1
CS010803000005,93,1
CS005412000085,98,1
CS035304000008,105,1
CS005315000108,115,1
CS037412000091,115,1
CS039413000007,115,1
CS003413000010,118,1
CS018613000019,118,1


---
> S-056: 顧客テーブル（customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [649]:
%%sql
SELECT customer_id, birth_day, 
    CASE WHEN age >= 60 THEN 60 ELSE TRUNC(age, -1) END AS 年代
FROM customer
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,birth_day,年代
CS021313000114,1981-04-29,30
CS037613000071,1952-04-01,60
CS031415000172,1976-10-04,40
CS028811000001,1933-03-27,60
CS001215000145,1995-03-29,20
CS020401000016,1974-09-15,40
CS015414000103,1977-08-09,40
CS029403000008,1973-08-17,40
CS015804000004,1931-05-02,60
CS033513000180,1962-07-11,50


---
> S-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [28]:
%%sql
SELECT customer_id, birth_day,
    CASE 
        WHEN gender_cd = '0' THEN 'M'
        WHEN gender_cd = '1' THEN 'F'
    ELSE 'U' END ||
    CASE WHEN age >= 60 THEN 60 
    ELSE TRUNC(age, -1) END AS 性別・年代
FROM customer
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,birth_day,性別・年代
CS021313000114,1981-04-29,F30
CS037613000071,1952-04-01,U60
CS031415000172,1976-10-04,F40
CS028811000001,1933-03-27,F60
CS001215000145,1995-03-29,F20
CS020401000016,1974-09-15,M40
CS015414000103,1977-08-09,F40
CS029403000008,1973-08-17,M40
CS015804000004,1931-05-02,M60
CS033513000180,1962-07-11,F50


---
> S-058: 顧客テーブル（customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [651]:
%%sql
SELECT customer_id,
    CASE WHEN gender_cd = '0' THEN 1 ELSE 0 END AS 男性,
    CASE WHEN gender_cd = '1' THEN 1 ELSE 0 END AS 女性,
    CASE WHEN gender_cd = '9' THEN 1 ELSE 0 END AS 不明
FROM customer
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,男性,女性,不明
CS021313000114,0,1,0
CS037613000071,0,0,1
CS031415000172,0,1,0
CS028811000001,0,1,0
CS001215000145,0,1,0
CS020401000016,1,0,0
CS015414000103,0,1,0
CS029403000008,1,0,0
CS015804000004,1,0,0
CS033513000180,0,1,0


---
> S-059: レシート明細テーブル（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [654]:
%%sql
-- 会員の合計金額テーブルを先に作成。
WITH mem_table AS (
    SELECT customer_id, SUM(amount) AS sum_amount
    FROM receipt
    WHERE customer_id !~ '^Z.*'
    GROUP BY customer_id
)
SELECT customer_id, sum_amount, (sum_amount - average) / standard_dev AS 標準得点
-- 統計における標準化の一般式は、(元の値 - 平均値) / 標準偏差
FROM mem_table
-- 計算に利用するため、平均値・標準偏差をCROSS JOINでテーブルに付与
CROSS JOIN (SELECT AVG(sum_amount) AS average, STDDEV(sum_amount) AS standard_dev FROM mem_table) AS index
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,標準得点
CS001311000059,2302,-0.09032402671702291270
CS004614000122,248,-0.84528400025253654164
CS003512000043,298,-0.82690620148098070841
CS011615000061,246,-0.84601911220339877497
CS029212000033,3604,0.38823385329429098451
CS007515000119,7157,1.6941602340010485
CS034515000123,3699,0.42315167096024706764
CS004315000058,490,-0.75633545419820630882
CS026414000014,6671,1.5155280299415258
CS001615000099,768,-0.65415489302835587608


---
> S-060: レシート明細テーブル（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [655]:
%%sql
WITH mem_table AS (
    SELECT customer_id, SUM(amount) AS sum_amount
    FROM receipt
    WHERE customer_id !~ '^Z.*'
    GROUP BY customer_id
)
SELECT customer_id, sum_amount, CAST((sum_amount - minimum) AS numeric) / CAST((maximum - minimum) AS numeric) AS 正規化点数
-- integerからnumeric等へデータ型の変換が必要。
FROM mem_table
CROSS JOIN (SELECT MIN(sum_amount) AS minimum, MAX(sum_amount) AS maximum FROM mem_table) AS index
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,正規化点数
CS001311000059,2302,0.09697601668404588113
CS004614000122,248,0.00773375043448036149
CS003512000043,298,0.00990615224191866528
CS011615000061,246,0.00764685436218282934
CS029212000033,3604,0.15354535974973931178
CS007515000119,7157,0.30791623218630517901
CS034515000123,3699,0.15767292318387208898
CS004315000058,490,0.01824817518248175182
CS026414000014,6671,0.28680048661800486618
CS001615000099,768,0.03032672923183872089


---
> S-061: レシート明細テーブル（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [656]:
%%sql
SELECT customer_id, sum_amount, LOG(sum_amount)
FROM (
    SELECT customer_id, SUM(amount) AS sum_amount
    FROM receipt
    WHERE customer_id !~ '^Z.*'
    GROUP BY customer_id
) AS mem_table
LIMIT 10


 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,log
CS001311000059,2302,3.362105319293773
CS004614000122,248,2.3944516808262164
CS003512000043,298,2.4742162640762553
CS011615000061,246,2.3909351071033793
CS029212000033,3604,3.5567847823070253
CS007515000119,7157,3.8547310172139424
CS034515000123,3699,3.568084331315394
CS004315000058,490,2.690196080028514
CS026414000014,6671,3.824190940652583
CS001615000099,768,2.885361220031512


---
> S-062: レシート明細テーブル（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ（ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること）。結果は10件表示させれば良い。

In [657]:
%%sql
SELECT customer_id, sum_amount, LN(sum_amount)
FROM (
    SELECT customer_id, SUM(amount) AS sum_amount
    FROM receipt
    WHERE customer_id !~ '^Z.*'
    GROUP BY customer_id
) AS mem_table
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,ln
CS001311000059,2302,7.741533589281828
CS004614000122,248,5.5134287461649825
CS003512000043,298,5.697093486505405
CS011615000061,246,5.5053315359323625
CS029212000033,3604,8.189799618728228
CS007515000119,7157,8.875846177738598
CS034515000123,3699,8.215817791832453
CS004315000058,490,6.194405391104672
CS026414000014,6671,8.805525052709516
CS001615000099,768,6.643789733147672


---
> S-063: 商品テーブル（product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [658]:
%%sql
SELECT product_cd, unit_price - unit_cost AS 利益額
FROM product
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,利益額
P040101001,49
P040101002,54
P040101003,57
P040101004,62
P040101005,67
P040101006,74
P040101007,84
P040101008,105
P040101009,124
P040101010,145


---
> S-064: 商品テーブル（product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。 ただし、単価と原価にはNULLが存在することに注意せよ。

In [659]:
%%sql
SELECT CAST(SUM(unit_price - unit_cost) AS numeric) / CAST(SUM(unit_price) AS numeric) AS 平均利益率
FROM product
WHERE unit_price IS NOT NULL AND unit_cost IS NOT NULL

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


平均利益率
0.24935966780792636459


---
> S-065: 商品テーブル（product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [660]:
%%sql
SELECT 
    product_cd, 
    TRUNC(unit_cost / 0.7, 0) AS 想定単価, 
    CAST(TRUNC(unit_cost / 0.7, 0) - unit_cost AS numeric) / CAST(TRUNC(unit_cost / 0.7, 0) AS numeric) AS 想定利益率
FROM product
WHERE unit_cost IS NOT NULL
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,想定単価,想定利益率
P040101001,212,0.29716981132075471698
P040101002,234,0.29914529914529914530
P040101003,247,0.29959514170040485830
P040101004,265,0.29811320754716981132
P040101005,287,0.29965156794425087108
P040101006,320,0.30000000000000000000
P040101007,362,0.29834254143646408840
P040101008,450,0.30000000000000000000
P040101009,534,0.29962546816479400749
P040101010,621,0.29951690821256038647


---
> S-066: 商品テーブル（product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [661]:
%%sql
SELECT 
    product_cd, 
    ROUND(unit_cost / 0.7, 0) AS 想定単価, 
    CAST(ROUND(unit_cost / 0.7, 0) - unit_cost AS numeric) / CAST(ROUND(unit_cost / 0.7, 0) AS numeric) AS 想定利益率
FROM product
WHERE unit_cost IS NOT NULL
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,想定単価,想定利益率
P040101001,213,0.30046948356807511737
P040101002,234,0.29914529914529914530
P040101003,247,0.29959514170040485830
P040101004,266,0.30075187969924812030
P040101005,287,0.29965156794425087108
P040101006,320,0.30000000000000000000
P040101007,363,0.30027548209366391185
P040101008,450,0.30000000000000000000
P040101009,534,0.29962546816479400749
P040101010,621,0.29951690821256038647


---
> S-067: 商品テーブル（product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [246]:
%%sql
SELECT 
    product_cd, 
    suggested_price AS 想定単価,
    CAST(suggested_price - unit_cost AS numeric) / CAST(suggested_price AS numeric) AS 想定利益率
FROM (
    SELECT product_cd, unit_cost, 
    -- 小数点以下が0の場合のみ切り上げなし、それ以外は切り捨て後に+1することで切り上げを実装。
        CASE
            WHEN (unit_cost / 0.7) - TRUNC(unit_cost / 0.7, 0) = 0 THEN TRUNC(unit_cost / 0.7, 0)
        ELSE TRUNC(unit_cost / 0.7, 0) + 1 END AS suggested_price
    FROM product
    WHERE unit_cost IS NOT NULL
) AS suggested_price_table
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,suggested_price,?column?
P040101001,213,0.30046948356807511737
P040101002,235,0.30212765957446808511
P040101003,248,0.30241935483870967742
P040101004,266,0.30075187969924812030
P040101005,288,0.30208333333333333333
P040101006,320,0.30000000000000000000
P040101007,363,0.30027548209366391185
P040101008,450,0.30000000000000000000
P040101009,535,0.30093457943925233645
P040101010,622,0.30064308681672025723


---
> S-068: 商品テーブル（product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [253]:
%%sql
SELECT product_cd, TRUNC(unit_price * 1.1, 0)
FROM product
WHERE unit_price IS NOT NULL
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,trunc
P040101001,217
P040101002,239
P040101003,253
P040101004,272
P040101005,294
P040101006,327
P040101007,371
P040101008,462
P040101009,547
P040101010,638


---
> S-069: レシート明細テーブル（receipt）と商品テーブル（product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [663]:
%%sql
-- 先にWITH句にてカテゴリ07の合計金額と全体の合計金額を用意する。
WITH cat07customer_table AS(
        SELECT customer_id, SUM(amount) AS sum_cat07
        FROM receipt LEFT OUTER JOIN product ON receipt.product_cd = product.product_cd
        WHERE category_major_cd = '07'
        GROUP BY customer_id, category_major_cd
),
    allcustomer_table AS(
        SELECT customer_id, SUM(amount) AS sum_all
        FROM receipt
        GROUP BY customer_id
)
SELECT cat07customer_table.customer_id, sum_all, sum_cat07, CAST(sum_cat07 AS numeric) / CAST(sum_all AS numeric) AS "07_瓶詰缶詰の割合"
FROM cat07customer_table LEFT OUTER JOIN allcustomer_table 
ON cat07customer_table.customer_id = allcustomer_table.customer_id
LIMIT 10


 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_all,sum_cat07,07_瓶詰缶詰の割合
CS017513000024,484,98,0.20247933884297520661
CS012513000141,1206,272,0.22553897180762852405
CS027414000018,3558,1594,0.44800449690837549185
CS015415000080,7685,3069,0.39934938191281717632
CS029315000039,2310,2200,0.95238095238095238095
CS002415000454,968,378,0.39049586776859504132
CS032314000002,3286,1791,0.54503956177723676202
CS009515000147,8364,6007,0.71819703491152558584
CS018215000027,5564,3846,0.69122933141624730410
CS035315000141,188,100,0.53191489361702127660


---
> S-070: レシート明細テーブル（receipt）の売上日（sales_ymd）に対し、顧客テーブル（customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [664]:
%%sql
SELECT receipt.customer_id, sales_ymd, application_date,
    CAST(CAST(sales_ymd AS varchar) AS date) - CAST(application_date AS date) AS 経過日数
FROM receipt LEFT OUTER JOIN customer ON receipt.customer_id = customer.customer_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sales_ymd,application_date,経過日数
CS006214000001,20181103,20150201,1371
CS008415000097,20181118,20150322,1337
CS028414000014,20170712,20150711,732
ZZ000000000000,20190205,None,None
CS025415000050,20180821,20160131,933
CS003515000195,20190605,20150306,1552
CS024514000042,20181205,20151010,1152
CS040415000178,20190922,20150627,1548
ZZ000000000000,20170504,None,None
CS027514000015,20191010,20151101,1439


---
> S-071: レシート明細テーブル（receipt）の売上日（sales_ymd）に対し、顧客テーブル（customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [666]:
%%sql
SELECT receipt.customer_id, sales_ymd, application_date,
    --DATE_PARTの年数差*12したものにDATE_PARTの月数差を加え、経過月数を計算する。
    DATE_PART('year', AGE(CAST(CAST(sales_ymd AS varchar) AS date), CAST(application_date AS date))) * 12 
    + DATE_PART('month', AGE(CAST(CAST(sales_ymd AS varchar) AS date), CAST(application_date AS date))) AS 経過月数
FROM receipt LEFT OUTER JOIN customer ON receipt.customer_id = customer.customer_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sales_ymd,application_date,経過月数
CS006214000001,20181103,20150201,45.0
CS008415000097,20181118,20150322,43.0
CS028414000014,20170712,20150711,24.0
ZZ000000000000,20190205,None,None
CS025415000050,20180821,20160131,30.0
CS003515000195,20190605,20150306,50.0
CS024514000042,20181205,20151010,37.0
CS040415000178,20190922,20150627,50.0
ZZ000000000000,20170504,None,None
CS027514000015,20191010,20151101,47.0


---
> S-072: レシート明細テーブル（receipt）の売上日（sales_ymd）に対し、顧客テーブル（customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [667]:
%%sql
SELECT receipt.customer_id, sales_ymd, application_date,
    DATE_PART('year', AGE(CAST(CAST(sales_ymd AS varchar) AS date), CAST(application_date AS date))) AS 経過年数
FROM receipt LEFT OUTER JOIN customer ON receipt.customer_id = customer.customer_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sales_ymd,application_date,経過年数
CS006214000001,20181103,20150201,3.0
CS008415000097,20181118,20150322,3.0
CS028414000014,20170712,20150711,2.0
ZZ000000000000,20190205,None,None
CS025415000050,20180821,20160131,2.0
CS003515000195,20190605,20150306,4.0
CS024514000042,20181205,20151010,3.0
CS040415000178,20190922,20150627,4.0
ZZ000000000000,20170504,None,None
CS027514000015,20191010,20151101,3.0


---
> S-073: レシート明細テーブル（receipt）の売上日（sales_ymd）に対し、顧客テーブル（customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [668]:
%%sql
SELECT receipt.customer_id, sales_ymd, application_date,
    DATE_PART('epoch', AGE(CAST(CAST(sales_ymd AS varchar) AS date), CAST(application_date AS date))) AS 経過時間（エポック秒）
FROM receipt LEFT OUTER JOIN customer ON receipt.customer_id = customer.customer_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sales_ymd,application_date,経過時間（エポック秒）
CS006214000001,20181103,20150201,118173600.0
CS008415000097,20181118,20150322,115149600.0
CS028414000014,20170712,20150711,63201600.0
ZZ000000000000,20190205,None,None
CS025415000050,20180821,20160131,80481600.0
CS003515000195,20190605,20150306,134006400.0
CS024514000042,20181205,20151010,99511200.0
CS040415000178,20190922,20150627,133574400.0
ZZ000000000000,20170504,None,None
CS027514000015,20191010,20151101,123962400.0


---
> S-074: レシート明細テーブル（receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [670]:
%%sql
SELECT sales_ymd, sales_ymd - DOW_num AS 購入した週の月曜日, DOW_num AS 経過日数
FROM (
    SELECT sales_ymd, 
    -- DATE_PARTのdowは日曜起点のため、日曜は+7, その他は-1することで月曜起点に変換。また、varcharで出力される？のでintegerに変換。
        CAST(
            CASE WHEN DATE_PART('dow', CAST(CAST(sales_ymd AS varchar) AS date)) = 0 THEN 7
            ELSE DATE_PART('dow', CAST(CAST(sales_ymd AS varchar) AS date)) - 1 END 
            AS integer
        ) AS DOW_num
    FROM receipt
    ) AS dow_table
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,購入した週の月曜日,経過日数
20181103,20181098,5
20181118,20181111,7
20170712,20170710,2
20190205,20190204,1
20180821,20180820,1
20190605,20190603,2
20181205,20181203,2
20190922,20190915,7
20170504,20170501,3
20191010,20191007,3


---
> S-075: 顧客テーブル（customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。 

In [671]:
%%sql
SELECT * FROM customer
WHERE customer_id IN (
    SELECT customer_id
    FROM (
        SELECT RANDOM() AS random_num, * FROM customer
    ) AS random_num_table
    WHERE random_num >= 0.99
)
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS040313000013,五十嵐 美帆,1,1980-10-07,38,226-0026,神奈川県横浜市緑区長津田町**********,S14040,20151029,0-00000000-0
CS004315000058,宇多田 文世,1,1984-01-28,35,165-0027,東京都中野区野方**********,S13004,20151101,4-20080517-1
CS023304000006,黒木 聖陽,0,1985-05-02,33,212-0006,神奈川県川崎市幸区遠藤町**********,S14023,20171004,0-00000000-0
CS019615000001,藤川 花,9,1955-09-23,63,176-0003,東京都練馬区羽沢**********,S13019,20150630,0-00000000-0
CS017613000127,大後 結子,1,1953-05-25,65,166-0000,東京都杉並区**********,S13017,20170308,0-00000000-0
CS032315000152,板倉 貴美子,1,1979-04-30,39,144-0055,東京都大田区仲六郷**********,S13032,20151112,0-00000000-0
CS037514000012,秋本 康文,9,1961-08-10,57,135-0016,東京都江東区東陽**********,S13037,20150413,9-20100815-9
CS039315000003,春日 陽子,1,1983-06-16,35,167-0032,東京都杉並区天沼**********,S13039,20150812,6-20080924-4
CS012513000141,中田 文世,1,1961-10-10,57,231-0844,神奈川県横浜市中区西之谷町**********,S14012,20150310,7-20100322-9
CS008613000035,大河内 恵麻,1,1954-02-01,65,157-0072,東京都世田谷区祖師谷**********,S13008,20150731,0-00000000-0


---
> S-076:  顧客テーブル（customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [8]:
%%sql
-- あらかじめRANDOMを持つテーブルを3つ用意する。
WITH 
    randum_num_table1 AS (SELECT RANDOM() AS random_num, customer_id, gender_cd FROM customer),
    randum_num_table2 AS (SELECT RANDOM() AS random_num, customer_id, gender_cd FROM customer),
    randum_num_table3 AS (SELECT RANDOM() AS random_num, customer_id, gender_cd FROM customer)
SELECT 
    SUM(CASE WHEN table1.random_num >= 0.9 AND table1.gender_cd = '0' THEN 1 ELSE 0 END) AS "男性_10%",
    SUM(CASE WHEN table2.random_num >= 0.9 AND table2.gender_cd = '1' THEN 1 ELSE 0 END) AS "女性_10%",
    SUM(CASE WHEN table3.random_num >= 0.9 AND table3.gender_cd = '9' THEN 1 ELSE 0 END) AS "不明_10%"
FROM randum_num_table1 table1 
INNER JOIN randum_num_table2 table2 ON table1.customer_id = table2.customer_id
INNER JOIN randum_num_table3 table3 ON table1.customer_id = table3.customer_id

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


男性_10%,女性_10%,不明_10%
299,1812,113


---
> S-077: レシート明細テーブル（receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [9]:
%%sql
WITH mem_table AS (
    SELECT customer_id, SUM(amount) AS sum_amount FROM receipt
    WHERE customer_id !~ '^Z.*'
    GROUP BY customer_id
)
SELECT customer_id, sum_amount
FROM mem_table
CROSS JOIN (SELECT AVG(sum_amount) AS average, STDDEV(sum_amount) AS standard_dev FROM mem_table) AS index
WHERE sum_amount >= average + 3 * standard_dev OR sum_amount <= average - 3 * standard_dev
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount
CS021514000008,12839
CS022515000065,12903
CS007514000094,15735
CS015515000034,15300
CS026414000039,12944
CS018414000065,12313
CS011415000006,16094
CS011414000012,11432
CS021515000172,13974
CS031414000028,11743


---
> S-078: レシート明細テーブル（receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [10]:
%%sql
WITH mem_table AS (
    SELECT customer_id, SUM(amount) AS sum_amount FROM receipt
    WHERE customer_id !~ '^Z.*'
    GROUP BY customer_id
)
SELECT customer_id, sum_amount
FROM mem_table
CROSS JOIN (
    SELECT 
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY sum_amount) AS Q1,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY sum_amount) AS Q3
    FROM mem_table
) AS index
WHERE sum_amount >= (Q3 + 1.5 * (Q3 - Q1)) OR sum_amount <= (Q3 - 1.5 * (Q3 - Q1))
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount
CS013415000226,8362
CS011415000041,9454
CS014514000004,8872
CS021514000008,12839
CS014515000007,9763
CS040415000220,10158
CS028415000161,8465
CS034515000173,10074
CS022515000065,12903
CS007514000094,15735


---
> S-079: 商品テーブル（product）の各項目に対し、欠損数を確認せよ。

In [11]:
%%sql
SELECT
    SUM(CASE WHEN product_cd IS NULL THEN 1 ELSE 0 END) AS product_code_NULL数,
    SUM(CASE WHEN category_major_cd IS NULL THEN 1 ELSE 0 END) AS category_major_cd_NULL数,
    SUM(CASE WHEN category_medium_cd IS NULL THEN 1 ELSE 0 END) AS category_medium_cd_NULL数,
    SUM(CASE WHEN category_small_cd IS NULL THEN 1 ELSE 0 END) AS category_small_cd_NULL数,
    SUM(CASE WHEN unit_price IS NULL THEN 1 ELSE 0 END) AS unit_price_NULL数,
    SUM(CASE WHEN unit_cost IS NULL THEN 1 ELSE 0 END) AS unit_cost_NULL数
FROM product

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


product_code_null数,category_major_cd_null数,category_medium_cd_null数,category_small_cd_null数,unit_price_null数,unit_cost_null数
0,0,0,0,7,7


---
> S-080: 商品テーブル（product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなproduct_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [360]:
%%sql
-- CREATE TABLE product_1 AS SELECT * FROM product
/*
DELETE FROM product_1
WHERE product_cd IS NULL
    OR category_major_cd IS NULL
    OR category_medium_cd IS NULL
    OR category_small_cd IS NULL
    OR unit_price IS NULL
    OR unit_cost IS NULL
*/

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


count
10023


---
> S-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなproduct_2を作成せよ。なお、平均値について1円未満は四捨五入とする。補完実施後、各項目について欠損が生じていないことも確認すること。

In [373]:
%%sql
-- CREATE TABLE product_2 AS SELECT * FROM product
/*
UPDATE product_2 SET
    unit_price = (SELECT ROUND(AVG(unit_price), 0) FROM product_2 WHERE unit_price IS NOT NULL)
WHERE unit_price IS NULL
*/
/*
UPDATE product_2 SET
    unit_cost = (SELECT ROUND(AVG(unit_cost), 0) FROM product_2 WHERE unit_cost IS NOT NULL)
WHERE unit_cost IS NULL
*/

SELECT
    SUM(CASE WHEN unit_price IS NULL THEN 1 ELSE 0 END) AS unit_price_NULL数,
    SUM(CASE WHEN unit_cost IS NULL THEN 1 ELSE 0 END) AS unit_cost_NULL数
FROM product_2

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


unit_price_null数,unit_cost_null数
0,0


---
> S-082:  単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなproduct_3を作成せよ。なお、中央値について1円未満は四捨五入とする。補完実施後、各項目について欠損が生じていないことも確認すること。

In [391]:
%%sql
-- CREATE TABLE product_3 AS SELECT * FROM product
/*
UPDATE product_3 SET
    unit_price = (SELECT ROUND(CAST(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY unit_price) AS numeric), 0) FROM product_3 WHERE unit_price IS NOT NULL)
WHERE unit_price IS NULL
*/
/*
UPDATE product_3 SET
    unit_cost = (SELECT ROUND(CAST(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY unit_cost) AS numeric), 0) FROM product_3 WHERE unit_cost IS NOT NULL)
WHERE unit_cost IS NULL
*/

SELECT
    SUM(CASE WHEN unit_price IS NULL THEN 1 ELSE 0 END) AS unit_price_NULL数,
    SUM(CASE WHEN unit_cost IS NULL THEN 1 ELSE 0 END) AS unit_cost_NULL数
FROM product_3


 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


unit_price_null数,unit_cost_null数
0,0


---
> S-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなproduct_4を作成せよ。なお、中央値について1円未満は四捨五入とする。補完実施後、各項目について欠損が生じていないことも確認すること。

In [415]:
%%sql
-- CREATE TABLE product_4 AS SELECT * FROM product
/*
UPDATE product_4 SET
unit_price = median_up_table.median_unit_price
FROM (
    SELECT 
        category_small_cd AS median_scd, 
        ROUND(CAST(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY unit_price) AS numeric), 0) AS median_unit_price
    FROM product_4
    GROUP BY category_small_cd
) AS median_up_table
WHERE unit_price IS NULL AND category_small_cd = median_scd
*/
/*
UPDATE product_4 SET
unit_cost = median_uc_table.median_unit_cost
FROM (
    SELECT 
        category_small_cd AS median_scd, 
        ROUND(CAST(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY unit_cost) AS numeric), 0) AS median_unit_cost
    FROM product_4
    GROUP BY category_small_cd
) AS median_uc_table
WHERE unit_cost IS NULL AND category_small_cd = median_scd
*/
SELECT
    SUM(CASE WHEN unit_price IS NULL THEN 1 ELSE 0 END) AS unit_price_NULL数,
    SUM(CASE WHEN unit_cost IS NULL THEN 1 ELSE 0 END) AS unit_cost_NULL数
FROM product_4

 * postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


unit_price_null数,unit_cost_null数
0,0


---
> S-084: 顧客テーブル（customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。

In [13]:
%%sql
SELECT customer.customer_id, CAST(amount_2019 AS numeric)/ CAST(sum_amount AS numeric) AS "2019年売上金額の割合"
FROM customer
INNER JOIN (
        SELECT customer_id, SUM(amount) AS sum_amount FROM receipt GROUP BY customer_id
    ) AS sum_amount_table
    ON customer.customer_id = sum_amount_table.customer_id
INNER JOIN (
        SELECT customer_id, SUM(amount) AS amount_2019 FROM receipt GROUP BY customer_id, LEFT(CAST(sales_ymd AS varchar), 4)
    ) AS amount_2019_table
    ON customer.customer_id = amount_2019_table.customer_id
WHERE CAST(amount_2019 AS numeric)/ CAST(sum_amount AS numeric) > 0
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,2019年売上金額の割合
CS014515000113,0.49046321525885558583
CS010415000255,0.68803687408338571129
CS023514000032,0.19893292682926829268
CS034515000077,0.34115805946791862285
CS032615000029,1.00000000000000000000
CS024415000152,0.83312577833125778331
CS005514000148,1.00000000000000000000
CS035215000028,0.67799309795593310327
CS020415000153,0.62631935702527355476
CS040415000070,0.53670530382859149982


---
> S-085: 顧客テーブル（customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用テーブル（geocode）を紐付け、新たなcustomer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。

In [17]:
%%sql
/*
CREATE TABLE customer_1 AS (
    SELECT 
        customer.*,
        avg_longitude AS longitude,
        avg_latitude AS latitude
    FROM customer
    LEFT OUTER JOIN (
        SELECT postal_cd, AVG(longitude) AS avg_longitude, AVG(latitude) AS avg_latitude
        FROM geocode
        GROUP BY postal_cd
    ) AS fixed_geocode
    ON customer.postal_cd = fixed_geocode.postal_cd
)
*/

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,longitude,latitude
CS037613000071,六角 雅彦,9,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.8350200000000000,35.6719300000000000
CS031415000172,宇多田 貴美子,1,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,139.6896500000000000,35.6737400000000000
CS028811000001,堀井 かおり,1,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,139.4836000000000000,35.3912500000000000
CS001215000145,田崎 美紀,1,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,139.7077500000000000,35.5408400000000000
CS015414000103,奥野 陽子,1,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B,139.8360100000000000,35.6781800000000000
CS015804000004,松谷 米蔵,0,1931-05-02,87,136-0073,東京都江東区北砂**********,S13015,20150607,0-00000000-0,139.8360100000000000,35.6781800000000000
CS007403000016,依田 満,0,1975-08-18,43,276-0022,千葉県八千代市上高野**********,S12007,20150914,0-00000000-0,140.1326000000000000,35.7326400000000000
CS035614000014,板倉 菜々美,1,1954-07-16,64,154-0015,東京都世田谷区桜新町**********,S13035,20150804,0-00000000-0,139.6429700000000000,35.6288900000000000
CS011215000048,芦田 沙耶,1,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9,139.6294600000000000,35.5537000000000000
CS040412000191,川井 郁恵,1,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4,139.5396400000000000,35.5276300000000000


---
> S-086: 前設問で作成した緯度経度つき顧客テーブル（customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗テーブル（store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [19]:
%%sql
SELECT 
    customer_id, 
    customer_1.address, 
    store.address, 
    6371 * ACOS(SIN(RADIANS(customer_1.latitude)) * SIN(RADIANS(store.latitude)) + COS(RADIANS(customer_1.latitude)) * COS(RADIANS(store.latitude)) * COS(RADIANS(customer_1.longitude) - RADIANS(store.longitude))) AS "距離(km)"
FROM customer_1 
LEFT OUTER JOIN store ON customer_1.application_store_cd = store.store_cd
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,address,address_1,距離(km)
CS037613000071,東京都江東区南砂**********,東京都江東区南砂一丁目,1.4511822099658445
CS031415000172,東京都渋谷区代々木**********,東京都渋谷区初台二丁目,0.4117334789298223
CS028811000001,神奈川県横浜市泉区和泉町**********,神奈川県横浜市瀬谷区二ツ橋町,8.065196026704987
CS001215000145,東京都大田区仲六郷**********,東京都大田区仲六郷二丁目,1.2684209720729687
CS015414000103,東京都江東区北砂**********,東京都江東区南砂二丁目,1.449673414532165
CS015804000004,東京都江東区北砂**********,東京都江東区南砂二丁目,1.449673414532165
CS007403000016,千葉県八千代市上高野**********,千葉県佐倉市上志津,1.9208032538419055
CS035614000014,東京都世田谷区桜新町**********,東京都世田谷区用賀四丁目,1.0214681484997588
CS011215000048,神奈川県横浜市港北区日吉本町**********,神奈川県横浜市港北区日吉本町四丁目,0.8182767808775093
CS040412000191,神奈川県横浜市緑区北八朔町**********,神奈川県横浜市緑区長津田みなみ台五丁目,3.6641233580823287


---
> S-087: 顧客テーブル（customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客テーブル（customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [464]:
%%sql
CREATE TABLE customer_u AS (
    SELECT 
        MIN(customer_id) AS customer_id, 
        sum_fixed_customer.customer_name, 
        MIN(gender_cd) AS gender_cd, 
        MIN(gender) AS gender, 
        MIN(birth_day) AS birth_day, 
        MIN(age) AS age,
        sum_fixed_customer.postal_cd,
        MIN(address) AS address,
        MIN(application_store_cd) AS application_store_cd,
        MIN(application_date) AS application_date,
        MIN(status_cd) AS status_cd,
        MIN(max_amount)
    FROM (
        SELECT * FROM customer
        LEFT OUTER JOIN (SELECT customer_id AS cid, SUM(amount) AS sum_amount FROM receipt GROUP BY customer_id) AS sum_amount_table
        ON customer.customer_id = sum_amount_table.cid    
    ) AS sum_fixed_customer
    LEFT OUTER JOIN (
        SELECT customer_name, postal_cd, MAX(sum_amount) AS max_amount
        FROM (
            SELECT * FROM customer
            LEFT OUTER JOIN (SELECT customer_id AS cid, SUM(amount) AS sum_amount FROM receipt GROUP BY customer_id) AS sum_amount_table
            ON customer.customer_id = sum_amount_table.cid
        ) AS sum_fixed_customer2
        GROUP BY customer_name, postal_cd
    ) AS max_fixed_customer
    ON sum_fixed_customer.customer_name = max_fixed_customer.customer_name
    AND sum_fixed_customer.postal_cd = max_fixed_customer.postal_cd
    WHERE sum_amount = max_amount OR sum_amount IS NULL
    GROUP BY sum_fixed_customer.customer_name, sum_fixed_customer.postal_cd
)

 * postgresql://padawan:***@db:5432/dsdojo_db
21941 rows affected.


[]

---
> S-088: 前設問で作成したデータを元に、顧客テーブルに統合名寄IDを付与したテーブル（customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
>- 重複していない顧客：顧客ID（customer_id）を設定
>- 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [21]:
%%sql
CREATE TABLE customer_n AS (
    SELECT customer_id_n, customer.*
    FROM customer
    LEFT OUTER JOIN (SELECT customer_id AS customer_id_n, customer_name AS u_name, postal_cd AS u_pcode FROM customer_u) AS c_u
    ON customer.customer_name = u_name AND customer.postal_cd = u_pcode
)

 * postgresql://padawan:***@db:5432/dsdojo_db
21971 rows affected.


[]

---
> S-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムに分割し、テーブルを作成せよ。

In [486]:
%%sql
/*
CREATE TABLE testtable_customer_n AS (
    SELECT * FROM customer_n
    WHERE customer_id IN (
            SELECT customer_id 
            FROM (
                    SELECT RANDOM() AS random_num, customer_id
                    FROM customer_n
                    WHERE customer_id IN (SELECT customer_id FROM receipt)
            ) AS for_testtable
            WHERE random_num < 0.2
        )
)
*/
/*
CREATE TABLE learntable_customer_n AS (
    SELECT * FROM (
        SELECT customer_id FROM customer_n WHERE customer_id IN (SELECT customer_id FROM receipt)
    ) AS shopped_member
    WHERE customer_id NOT IN (SELECT customer_id FROM testtable_customer_n)
)
*/

 * postgresql://padawan:***@db:5432/dsdojo_db
6574 rows affected.


[]

---
> S-090: レシート明細テーブル（receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3テーブルとしてセット作成せよ。データの持ち方は自由とする。

In [532]:
%%sql

/*
CREATE TABLE learntable_monthly_amount AS (
    SELECT sales_ym, sum_amount, test_num1, test_num2, test_num3
    FROM (
        SELECT ROW_NUMBER() OVER (ORDER BY LEFT(CAST(sales_ymd AS varchar), 6)) AS row_num, LEFT(CAST(sales_ymd AS varchar), 6) AS sales_ym, SUM(amount) AS sum_amount
        FROM receipt
        GROUP BY LEFT(CAST(sales_ymd AS varchar), 6)
    ) AS monthly_amount
    CROSS JOIN (
        SELECT 
            TRUNC(CAST(RANDOM() AS numeric) * 23, 0) + 1 AS learn_num, 
            TRUNC(CAST(RANDOM() AS numeric) * 29, 0) + 1 AS test_num1, 
            TRUNC(CAST(RANDOM() AS numeric) * 29, 0) + 1 AS test_num2, 
            TRUNC(CAST(RANDOM() AS numeric) * 29, 0) + 1 AS test_num3
    )AS random_num
    WHERE row_num BETWEEN learn_num AND learn_num + 11
)
*/
/*
CREATE TABLE testtable1_monthly_amount AS (
    SELECT sales_ym, sum_amount
    FROM (
        SELECT ROW_NUMBER() OVER (ORDER BY LEFT(CAST(sales_ymd AS varchar), 6)) AS row_num, LEFT(CAST(sales_ymd AS varchar), 6) AS sales_ym, SUM(amount) AS sum_amount
        FROM receipt
        GROUP BY LEFT(CAST(sales_ymd AS varchar), 6)
    ) AS monthly_amount
    WHERE row_num BETWEEN (SELECT MIN(test_num1) FROM learntable_monthly_amount) AND (SELECT MIN(test_num1) FROM learntable_monthly_amount) + 5
)
*/
/*
CREATE TABLE testtable2_monthly_amount AS (
    SELECT sales_ym, sum_amount
    FROM (
        SELECT ROW_NUMBER() OVER (ORDER BY LEFT(CAST(sales_ymd AS varchar), 6)) AS row_num, LEFT(CAST(sales_ymd AS varchar), 6) AS sales_ym, SUM(amount) AS sum_amount
        FROM receipt
        GROUP BY LEFT(CAST(sales_ymd AS varchar), 6)
    ) AS monthly_amount
    WHERE row_num BETWEEN (SELECT MIN(test_num2) FROM learntable_monthly_amount) AND (SELECT MIN(test_num2) FROM learntable_monthly_amount) + 5
)
*/
/*
CREATE TABLE testtable3_monthly_amount AS (
    SELECT sales_ym, sum_amount
    FROM (
        SELECT ROW_NUMBER() OVER (ORDER BY LEFT(CAST(sales_ymd AS varchar), 6)) AS row_num, LEFT(CAST(sales_ymd AS varchar), 6) AS sales_ym, SUM(amount) AS sum_amount
        FROM receipt
        GROUP BY LEFT(CAST(sales_ymd AS varchar), 6)
    ) AS monthly_amount
    WHERE row_num BETWEEN (SELECT MIN(test_num3) FROM learntable_monthly_amount) AND (SELECT MIN(test_num3) FROM learntable_monthly_amount) + 5
)
*/
-- ALTER TABLE learntable_monthly_amount DROP test_num1, DROP test_num2, DROP test_num3

 * postgresql://padawan:***@db:5432/dsdojo_db
Done.


[]

---
> S-091: 顧客テーブル（customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [560]:
%%sql
WITH shopped_member_1percent AS (
    SELECT customer_id
    FROM (
        SELECT RANDOM() AS random_num1, customer_id
        FROM customer
        WHERE customer_id IN (SELECT customer_id FROM receipt)
    ) AS shopped_mem
    WHERE random_num1 > 0.99
),
not_shopped_member AS (
    SELECT ROW_NUMBER() OVER (ORDER BY random_num2) AS row_num, customer_id
    FROM (
        SELECT RANDOM() AS random_num2, customer_id 
        FROM customer
        WHERE customer_id NOT IN (SELECT customer_id FROM receipt)
    ) AS not_shopped_mem
)
SELECT * FROM customer
WHERE customer_id IN (SELECT customer_id FROM shopped_member_1percent)
OR customer_id IN (
    SELECT customer_id FROM not_shopped_member
    WHERE row_num <= (SELECT COUNT(*) FROM shopped_member_1percent)
)


 * postgresql://padawan:***@db:5432/dsdojo_db
170 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS001715000106,内田 薫,1,女性,1943-08-05,75,146-0095,東京都大田区多摩川**********,S13001,20160317,0-00000000-0
CS017414000093,佐々木 涼,1,女性,1973-11-17,45,166-0011,東京都杉並区梅里**********,S13017,20160129,D-20101006-D
CS009415000117,川口 綾女,1,女性,1969-10-24,49,154-0014,東京都世田谷区新町**********,S13009,20150719,F-20101021-F
CS025411000053,田村 ちえみ,1,女性,1972-10-08,46,242-0011,神奈川県大和市深見**********,S14025,20160112,4-20100127-4
CS001304000018,杉本 人志,0,男性,1979-06-01,39,144-0055,東京都大田区仲六郷**********,S13001,20170522,3-20100212-2
CS037513000041,松田 エリカ,1,女性,1965-11-12,53,136-0075,東京都江東区新砂**********,S13037,20150623,0-00000000-0
CS005412000190,相原 法子,1,女性,1977-11-26,41,177-0041,東京都練馬区石神井町**********,S13005,20150215,7-20081027-8
CS001512000310,本間 薫,1,女性,1967-04-28,51,210-0804,神奈川県川崎市川崎区藤崎**********,S13001,20161221,0-00000000-0
CS017412000019,菊地 恵麻,1,女性,1969-04-08,49,167-0032,東京都杉並区天沼**********,S13017,20150205,5-20081027-3
CS043413000016,松原 咲,1,女性,1971-01-21,48,140-0004,東京都品川区南品川**********,S13043,20151120,4-20091025-4


---
> S-092: 顧客テーブル（customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [568]:
%%sql
/*
CREATE TABLE gender (
    gender_cd integer,
    gender char(2)
)
*/
-- INSERT INTO gender VALUES (0, '男性'), (1, '女性'), (9, '不明')
-- ALTER TABLE customer DROP COLUMN gender

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS021313000114,大野 あや子,1,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
CS037613000071,六角 雅彦,9,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS031415000172,宇多田 貴美子,1,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
CS028811000001,堀井 かおり,1,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
CS001215000145,田崎 美紀,1,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
CS020401000016,宮下 達士,0,1974-09-15,44,174-0065,東京都板橋区若木**********,S13020,20150225,0-00000000-0
CS015414000103,奥野 陽子,1,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
CS029403000008,釈 人志,0,1973-08-17,45,279-0003,千葉県浦安市海楽**********,S12029,20150515,0-00000000-0
CS015804000004,松谷 米蔵,0,1931-05-02,87,136-0073,東京都江東区北砂**********,S13015,20150607,0-00000000-0
CS033513000180,安斎 遥,1,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5


---
> S-093: 商品テーブル（product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリテーブル（category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品テーブルを作成せよ。

In [589]:
%%sql
/*
CREATE TABLE product_withname AS (
    SELECT product_cd, product.category_major_cd, category_major_name, product.category_medium_cd, category_medium_name,
    product.category_small_cd, category_small_name, unit_price, unit_cost
    FROM product LEFT OUTER JOIN category ON product.category_small_cd = category.category_small_cd
)
*/

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name,unit_price,unit_cost
P040101001,04,惣菜,0401,御飯類,040101,弁当類,198,149
P040101002,04,惣菜,0401,御飯類,040101,弁当類,218,164
P040101003,04,惣菜,0401,御飯類,040101,弁当類,230,173
P040101004,04,惣菜,0401,御飯類,040101,弁当類,248,186
P040101005,04,惣菜,0401,御飯類,040101,弁当類,268,201
P040101006,04,惣菜,0401,御飯類,040101,弁当類,298,224
P040101007,04,惣菜,0401,御飯類,040101,弁当類,338,254
P040101008,04,惣菜,0401,御飯類,040101,弁当類,420,315
P040101009,04,惣菜,0401,御飯類,040101,弁当類,498,374
P040101010,04,惣菜,0401,御飯類,040101,弁当類,580,435


---
> S-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。出力先のパスは'/tmp/data'を指定することでJupyterの/'work/data'と共有されるようになっている。なお、COPYコマンドの権限は付与済みである。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [603]:
%%sql
COPY product_withname TO '/tmp/data/product1.csv' WITH CSV HEADER ENCODING 'UTF8' DELIMITER ','

 * postgresql://padawan:***@db:5432/dsdojo_db
10030 rows affected.


[]

---
> S-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。出力先のパスは'/tmp/data'を指定することでJupyterの/'work/data'と共有されるようになっている。なお、COPYコマンドの権限は付与済みである。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはSJIS

In [579]:
%%sql
COPY product_withname TO '/tmp/data/product2.csv' WITH CSV HEADER ENCODING 'SJIS' DELIMITER ','

 * postgresql://padawan:***@db:5432/dsdojo_db
10030 rows affected.


[]

---
> S-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。出力先のパスは'/tmp/data'を指定することでJupyterの/'work/data'と共有されるようになっている。なお、COPYコマンドの権限は付与済みである。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [582]:
%%sql
COPY product_withname TO '/tmp/data/product3.csv' WITH CSV ENCODING 'UTF8' DELIMITER ','

 * postgresql://padawan:***@db:5432/dsdojo_db
10030 rows affected.


[]

---
> S-097: 先に作成した以下形式のファイルを読み込み、テーブルを作成せよ。また、先頭3件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [596]:
%%sql
-- CREATE TABLE product1 AS SELECT * FROM product_withname
-- TRUNCATE TABLE product1
-- COPY product1 FROM '/tmp/data/product1.csv' WITH CSV HEADER ENCODING 'UTF8' DELIMITER ','
SELECT * FROM product1 LIMIT 3

 * postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


product_cd,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name,unit_price,unit_cost
P040101001,04,惣菜,0401,御飯類,040101,弁当類,198,149
P040101002,04,惣菜,0401,御飯類,040101,弁当類,218,164
P040101003,04,惣菜,0401,御飯類,040101,弁当類,230,173


---
> S-098: 先に作成した以下形式のファイルを読み込み、テーブルを作成せよ。また、先頭3件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [600]:
%%sql
-- CREATE TABLE product3 AS SELECT * FROM product_withname
-- TRUNCATE TABLE product3
-- COPY product3 FROM '/tmp/data/product3.csv' WITH CSV ENCODING 'UTF8' DELIMITER ','
SELECT * FROM product3 LIMIT 3

 * postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


product_cd,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name,unit_price,unit_cost
P040101001,04,惣菜,0401,御飯類,040101,弁当類,198,149
P040101002,04,惣菜,0401,御飯類,040101,弁当類,218,164
P040101003,04,惣菜,0401,御飯類,040101,弁当類,230,173


---
> S-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。出力先のパスは'/tmp/data'を指定することでJupyterの/'work/data'と共有されるようになっている。なお、COPYコマンドの権限は付与済みである。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [604]:
%%sql
COPY product_withname TO '/tmp/data/product4.csv' WITH CSV HEADER ENCODING 'UTF8' DELIMITER E'\t'

 * postgresql://padawan:***@db:5432/dsdojo_db
10030 rows affected.


[]

---
> S-100: 先に作成した以下形式のファイルを読み込み、テーブルを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [608]:
%%sql
-- CREATE TABLE product4 AS SELECT * FROM product_withname
-- TRUNCATE TABLE product4
-- COPY product4 FROM '/tmp/data/product4.csv' WITH CSV HEADER ENCODING 'UTF8' DELIMITER E'\t'
-- SELECT * FROM product4 LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name,unit_price,unit_cost
P040101001,04,惣菜,0401,御飯類,040101,弁当類,198,149
P040101002,04,惣菜,0401,御飯類,040101,弁当類,218,164
P040101003,04,惣菜,0401,御飯類,040101,弁当類,230,173
P040101004,04,惣菜,0401,御飯類,040101,弁当類,248,186
P040101005,04,惣菜,0401,御飯類,040101,弁当類,268,201
P040101006,04,惣菜,0401,御飯類,040101,弁当類,298,224
P040101007,04,惣菜,0401,御飯類,040101,弁当類,338,254
P040101008,04,惣菜,0401,御飯類,040101,弁当類,420,315
P040101009,04,惣菜,0401,御飯類,040101,弁当類,498,374
P040101010,04,惣菜,0401,御飯類,040101,弁当類,580,435


# これで１００本終わりです。おつかれさまでした！